# Survey memory
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code demonstrating how to add "memories" of previous questions to other simulated survey questions, to allow comparison of responses to questions where memories of prior questions are not added.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Benefits of simulated surveys
Some benefits of using AI agents to answer surveys are that agents can be presented any number of questions to answer - tirelessly! - and that questions can be presented with or without information about other responses that have already been given. This allows us to easily explore how agents respond to the same questions with different contexts.

## No-memory default
By default, EDSL survey questions are administered asynchronously to the agents and language models added to the survey when it is run. This means that questions are answered independently of each other, unless other rules have been specified. This default behavior is useful for speed and minimizing token usage (not repeating prior question contexts unnecessarily).

However, in some cases we may want an agent to have the context of some or all prior questions when answering a given question (e.g., to immitate a human's experience answering a survey). Or, we may simply want to explore the potential impact of prior question contexts by running new questions with and without memories of them.

## How it works
EDSL has several options for adding survey memories:

* `set_full_memory_mode()` adds a memory of all prior questions and answers to each new question
* `set_lagged_memory(n)` adds a memory of a specified number of prior questions and answers to each new question
* `add_targeted_memory(<new_question>, <prior_question>)` adds a memory of specific prior question to a specific new question
* `add_memory_collection(<new_question>, [<list_of_prior_questions>])` adds memories of a set of prior questions and answers to a specific new question

[Learn more about memory rules](https://docs.expectedparrot.com/en/latest/surveys.html#question-memory).

## Creating questions
Here we create a survey of free text and multiple choice questions; [learn more about other question types](https://docs.expectedparrot.com/en/latest/questions.html) to choose from. Note that we include copies of questions in order for the results to show side-by-side responses where we have and have not added memories of prior questions:

In [1]:
from edsl import QuestionFreeText, QuestionMultipleChoice, Survey

In [2]:
q_credit = QuestionFreeText(
    question_name="credit",
    question_text="""Have you heard about the phenomenon of hearing about an idea and 
    then believing that you are responsible for it?""",
)

q_ideas_with_credit = QuestionFreeText(
    question_name="ideas_with_credit",
    question_text="What's an original idea for an application for large language models.",
)

q_ideas_no_credit = QuestionFreeText(
    question_name="ideas_no_credit",
    question_text="What's an original idea for an application for large language models.",
)

q_original_with_credit = QuestionMultipleChoice(
    question_name="original_with_credit",
    question_text="The idea you just described - is it original?",
    question_options=[
        "Yes, I thought of it myself.",
        "I may have heard about it before.",
        "No, I have heard about it before.",
        "I do not know.",
    ],
)

q_original_no_credit = QuestionMultipleChoice(
    question_name="original_no_credit",
    question_text="The idea you just described - is it original?",
    question_options=[
        "Yes, I thought of it myself.",
        "I may have heard about it before.",
        "No, I have heard about it before.",
        "I do not know.",
    ],
)

In [3]:
survey = Survey(
    [
        q_credit,
        q_ideas_with_credit,
        q_ideas_no_credit,
        q_original_with_credit,
        q_original_no_credit,
    ]
)

## Targeted memories
Here we add memories so that the results will include (1) a response to the "original" question where the "idea" question included the context of the "credit" question and (2) a response to the "original" question where the "idea" question did *not* include the context of the "credit" question:

In [4]:
survey = (
    survey
    # Adding a memory of "credit" to the first iteration of "ideas"
    .add_targeted_memory(q_ideas_with_credit, q_credit)
    # Adding memories of both "credit" and "ideas" to the first iteration of "original"
    .add_memory_collection(q_original_with_credit, [q_credit, q_ideas_with_credit])
    # Adding a memory of "ideas" where "credit" was not included to the second iteration of "original"
    .add_targeted_memory(q_original_no_credit, q_ideas_no_credit)
)

In [5]:
results = survey.run()

In [6]:
results.select(
    "ideas_with_credit", "original_with_credit", "ideas_no_credit", "original_no_credit"
).print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                     ┃ answer                    ┃ answer                     ┃ answer                    ┃
┃ .ideas_with_credit         ┃ .original_with_credit     ┃ .ideas_no_credit           ┃ .original_no_credit       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ An original idea for an    │ Yes, I thought of it      │ Sure, how about an         │ Yes, I thought of it      │
│ application using large    │ myself.                   │ "Interactive Storytelling  │ myself.                   │
│ language models could be a │                           │ Companion"? This           │                           │
│ "Creative Writing          │                           │ application would leverage │                           │
│ Collaborator" app. This    │                           │ large language models to   │                           │
│ app would serve as a       │                           │ create a dynamic,          │                           │
│ dynamic writing partner    │                           │ collaborative storytelling │                           │
│ for authors,               │                           │ experience. Users could    │                           │
│ screenwriters, and poets.  │                           │ start a story and at any   │                           │
│ Here’s how it could work:  │                           │ point, ask the model to    │                           │
│                            │                           │ continue, introduce new    │                           │
│ 1. **Idea Generation**:    │                           │ characters, or even change │                           │
│ Users can input a basic    │                           │ the setting. The model     │                           │
│ premise or a few keywords, │                           │ could also suggest plot    │                           │
│ and the app generates a    │                           │ twists or dialogue         │                           │
│ variety of plot ideas,     │                           │ options, making the        │                           │
│ character sketches, or     │                           │ storytelling process more  │                           │
│ thematic suggestions.      │                           │ engaging and creative.     │                           │
│                            │                           │                            │                           │
│ 2. **Real-time             │                           │ Features could include:    │                           │
│ Collaboration**: As the    │                           │ 1. **Genre Selection**:    │                           │
│ user writes, the app can   │                           │ Users can choose the genre │                           │
│ offer suggestions for      │                           │ (e.g., fantasy, sci-fi,    │                           │
│ dialogue, plot twists, or  │                           │ mystery) and the model     │                           │
│ descriptive language,      │                           │ will tailor its responses  │                           │
│ adapting to the user's     │                           │ accordingly.               │                           │
│ style and preferences over │                           │ 2. **Character             │                           │
│ time.                      │                           │ Development**: The model   │                           │
│                            │                           │ can help flesh out         │                           │
│ 3. **Editing and           │                           │ characters by providing    │                           │
│ Feedback**: The app can    │                          

We can examine the question prompts to verify how the rules were applied. To see a list of all the components of the results that can be directly accessed:

In [7]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.credit',
 'answer.ideas_no_credit',
 'answer.ideas_with_credit',
 'answer.original_no_credit',
 'answer.original_with_credit',
 'comment.credit_comment',
 'comment.ideas_no_credit_comment',
 'comment.ideas_with_credit_comment',
 'comment.original_no_credit_comment',
 'comment.original_with_credit_comment',
 'generated_tokens.credit_generated_tokens',
 'generated_tokens.ideas_no_credit_generated_tokens',
 'generated_tokens.ideas_with_credit_generated_tokens',
 'generated_tokens.original_no_credit_generated_tokens',
 'generated_tokens.original_with_credit_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.credit_system_prompt',
 'prompt.credit_user_prompt',
 'prompt.ideas_no_credit_system_prompt',
 'prompt.ideas_no_credit_user_prompt',
 'prompt.ideas_with_credi

Here we inspect the system (agent) and user (question) prompts for the "credit" question:

In [8]:
results.select("credit_system_prompt", "credit_user_prompt").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .credit_system_prompt                                  ┃ .credit_user_prompt                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ You are answering questions as if you were a human. Do │ Have you heard about the phenomenon of hearing about   │
│ not break character. You are an agent with the         │ an idea and                                            │
│ following persona:                                     │     then believing that you are responsible for it?    │
│ {}                                                     │                                                        │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

Here we can verify that we included the context of the "credit" question in the "_with_memory" iteration of the "ideas" question:

In [9]:
results.select(
    "ideas_with_credit_user_prompt", "original_with_credit_user_prompt"
).print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .ideas_with_credit_user_prompt                         ┃ .original_with_credit_user_prompt                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ What's an original idea for an application for large   │                                                        │
│ language models.                                       │ The idea you just described - is it original?          │
│         Before the question you are now answering, you │                                                        │
│ already answered the following question(s):            │                                                        │
│                 Question: Have you heard about the     │ Yes, I thought of it myself.                           │
│ phenomenon of hearing about an idea and                │                                                        │
│     then believing that you are responsible for it?    │ I may have heard about it before.                      │
│         Answer: Yes, that phenomenon is often referred │                                                        │
│ to as "cryptomnesia." It's when someone has a memory   │ No, I have heard about it before.                      │
│ that they believe is a new, original idea, but it's    │                                                        │
│ actually something they encountered previously and     │ I do not know.                                         │
│ forgot. This can happen unintentionally, and the       │                                                        │
│ person genuinely thinks they came up with the idea     │                                                        │
│ themselves. It's a fascinating aspect of how our       │ Only 1 option may be selected.                         │
│ memory works and highlights how our brains can         │                                                        │
│ sometimes blur the lines between original thought and  │ Respond only with a string corresponding to one of the │
│ recollection.                                          │ options.                                               │
│                                                        │                                                        │
│                                                        │                                                        │
│                                                        │ After the answer, you can put a comment explaining why │
│                                                        │ you chose that option on the next line.                │
│                                                        │         Before the question you are now answering, you │
│                                                        │ already answered the following question(s):            │
│                                                        │                 Question: Have you heard about the     │
│                                                        │ phenomenon of hearing about an idea and                │
│                                                        │     then believing that you are responsible for it?    │
│                                                        │         Answer: Yes, that phenomenon is often referred │
│                                                        │ to as "cryptomnesia." It's when someone has a memory   │
│                                                        │ that they believe is a new, original idea, but it's    │
│                                                        │ actually something they encountered previously and     │
│                                                       

In [10]:
results.select("ideas_no_credit_user_prompt", "original_no_credit_user_prompt").print(
    format="rich"
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                                                 ┃ prompt                                                 ┃
┃ .ideas_no_credit_user_prompt                           ┃ .original_no_credit_user_prompt                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ What's an original idea for an application for large   │                                                        │
│ language models.                                       │ The idea you just described - is it original?          │
│                                                        │                                                        │
│                                                        │                                                        │
│                                                        │ Yes, I thought of it myself.                           │
│                                                        │                                                        │
│                                                        │ I may have heard about it before.                      │
│                                                        │                                                        │
│                                                        │ No, I have heard about it before.                      │
│                                                        │                                                        │
│                                                        │ I do not know.                                         │
│                                                        │                                                        │
│                                                        │                                                        │
│                                                        │ Only 1 option may be selected.                         │
│                                                        │                                                        │
│                                                        │ Respond only with a string corresponding to one of the │
│                                                        │ options.                                               │
│                                                        │                                                        │
│                                                        │                                                        │
│                                                        │ After the answer, you can put a comment explaining why │
│                                                        │ you chose that option on the next line.                │
│                                                        │         Before the question you are now answering, you │
│                                                        │ already answered the following question(s):            │
│                                                        │                 Question: What's an original idea for  │
│                                                        │ an application for large language models.              │
│                                                        │         Answer: Sure, how about an "Interactive        │
│                                                        │ Storytelling Companion"? This application would        │
│                                                        │ leverage large language models to create a dynamic,    │
│                                                        │ collaborative storytelling experience. Users could     │
│                                                        │ start a story and at any point, ask the model to       │
│                                                       

## Full memory
Here we create another survey to demonstrate full memory mode:

In [11]:
q1 = QuestionFreeText(
    question_name="q1", question_text="How many hours a week do you work?"
)

q2 = QuestionFreeText(
    question_name="q2", question_text="Do you have a good work-life balance?"
)

q3 = QuestionFreeText(
    question_name="q3",
    question_text="What changes, if any, would you like to make in your work routines?",
)

Here we show the prompts if a lagged memory rule is applied:

In [12]:
survey = Survey([q1, q2, q3])

survey_full = survey.set_full_memory_mode()

results_full = survey_full.run()

results_full.select("q1_user_prompt", "q2_user_prompt", "q3_user_prompt").print(
    format="rich"
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ prompt                             ┃ prompt                              ┃ prompt                               ┃
┃ .q1_user_prompt                    ┃ .q2_user_prompt                     ┃ .q3_user_prompt                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ How many hours a week do you work? │ Do you have a good work-life        │ What changes, if any, would you like │
│                                    │ balance?                            │ to make in your work routines?       │
│                                    │         Before the question you are │         Before the question you are  │
│                                    │ now answering, you already answered │ now answering, you already answered  │
│                                    │ the following question(s):          │ the following question(s):           │
│                                    │                 Question: How many  │                 Question: How many   │
│                                    │ hours a week do you work?           │ hours a week do you work?            │
│                                    │         Answer: I typically work a  │         Answer: I typically work a   │
│                                    │ standard full-time schedule, which  │ standard full-time schedule, which   │
│                                    │ is around 40 hours a week. How      │ is around 40 hours a week. How about │
│                                    │ about you?                          │ you?                                 │
│                                    │                                     │                                      │
│                                    │                                     │  Prior questions and answers:        │
│                                    │                                     │ Question: Do you have a good         │
│                                    │                                     │ work-life balance?                   │
│                                    │                                     │         Answer: Yes, I do my best to │
│                                    │                                     │ maintain a good work-life balance. I │
│                                    │                                     │ find it's important to make time for │
│                                    │                                     │ hobbies, family, and relaxation      │
│                                    │                                     │ outside of work hours. How do you    │
│                                    │                                     │ manage your work-life balance?       │
└────────────────────────────────────┴─────────────────────────────────────┴──────────────────────────────────────┘

## Lagged memory
Here we show the prompts if a lagged memory rule is used instead (note that we need to re-construct the original survey to remove the full memory rule):

In [ ]:
survey = Survey([q1, q2, q3])

survey_lagged = survey.set_lagged_memory(1)

results_lagged = survey_lagged.run()

results_lagged.select("q1_user_prompt", "q2_user_prompt", "q3_user_prompt").print(
    format="rich"
)

## Other survey rules
See our [documentation page about other methods](https://docs.expectedparrot.com/en/latest/surveys.html) available for `Survey` objects, including methods for applying skip/stop and other logical rules.